In [1]:
from sklearn.preprocessing import LabelEncoder, MaxAbsScaler
from sklearn.model_selection import train_test_split
from joblib import dump
import pandas as pd
import numpy as np
import json

In [2]:
# depth of this file in the project
file_depth = '../..'

In [3]:
with open(file_depth + '/config/data_25m_config.json') as config_file:
    config = json.load(config_file)

In [4]:
new_context_data = config['new_context_data']
added_imdb_context = pd.read_csv(file_depth + new_context_data)
added_imdb_context

,userId,movieId,day,isWeekday,season,partOfDay,holiday,movieYear,titleType,isAdult,...,genreMystery,genreNews,genreReality-tv,genreRomance,genreSci-fi,genreShort,genreSport,genreThriller,genreWar,genreWestern
0,1,296,3,1,1,3,no_holiday,1994,movie,0,...,0,0,0,0,0,0,0,1,0,0
1,1,306,3,1,1,2,no_holiday,1994,movie,0,...,0,0,0,0,0,0,0,0,0,0
2,1,307,3,1,1,2,no_holiday,1993,movie,0,...,0,0,0,0,0,0,0,0,0,0
3,1,665,3,1,1,3,no_holiday,1995,movie,0,...,0,0,0,0,0,0,0,0,1,0
4,1,899,3,1,1,2,no_holiday,1952,movie,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24983464,162541,50872,2,1,1,4,no_holiday,2007,movie,0,...,0,0,0,0,0,0,0,0,0,0
24983465,162541,55768,2,1,1,4,no_holiday,2007,movie,0,...,0,0,0,0,0,0,0,0,0,0
24983466,162541,56176,2,1,1,4,no_holiday,2007,movie,0,...,0,0,0,0,0,0,0,0,0,0
24983467,162541,58559,2,1,1,4,no_holiday,2008,movie,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
added_imdb_context.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24983469 entries, 0 to 24983468
Data columns (total 41 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   userId            int64 
 1   movieId           int64 
 2   day               int64 
 3   isWeekday         int64 
 4   season            int64 
 5   partOfDay         int64 
 6   holiday           object
 7   movieYear         int64 
 8   titleType         object
 9   isAdult           int64 
 10  runtimeMinutes    int64 
 11  directors         object
 12  actor             object
 13  genreAction       int64 
 14  genreAdult        int64 
 15  genreAdventure    int64 
 16  genreAnimation    int64 
 17  genreBiography    int64 
 18  genreChildren     int64 
 19  genreComedy       int64 
 20  genreCrime        int64 
 21  genreDocumentary  int64 
 22  genreDrama        int64 
 23  genreFamily       int64 
 24  genreFantasy      int64 
 25  genreFilm-noir    int64 
 26  genreHistory      int64 
 27  genreHorro

In [6]:
all_non_null = []
all_non_null.append((added_imdb_context['runtimeMinutes'] == '\\N').any())
all_non_null.append((added_imdb_context['holiday'] == '\\N').any())
all_non_null.append((added_imdb_context['titleType'] == '\\N').any())
all_non_null.append((added_imdb_context['directors'] == '\\N').any())
all_non_null.append((added_imdb_context['actor'] == '\\N').any())

if any(item == True for item in all_non_null):
    print("There are null value in the columns with type object.")
else:
    print("There are no null value in the columns with type object.")

There are no null value in the columns with type object.


In [7]:
target_data = config['target_data']
ratings_target = pd.read_csv(file_depth + target_data)
ratings_target

,ratingId,rating
0,0,5.0
1,1,3.5
2,2,5.0
3,3,5.0
4,4,3.5
...,...,...
24983464,24983464,4.5
24983465,24983465,2.5
24983466,24983466,2.0
24983467,24983467,4.0


In [8]:
added_imdb_context['ratingId'] = added_imdb_context.index
added_imdb_context = added_imdb_context.merge(ratings_target, on='ratingId', how='left')
del ratings_target
added_imdb_context = added_imdb_context.drop(['ratingId'], axis=1)
added_imdb_context

,userId,movieId,day,isWeekday,season,partOfDay,holiday,movieYear,titleType,isAdult,...,genreNews,genreReality-tv,genreRomance,genreSci-fi,genreShort,genreSport,genreThriller,genreWar,genreWestern,rating
0,1,296,3,1,1,3,no_holiday,1994,movie,0,...,0,0,0,0,0,0,1,0,0,5.0
1,1,306,3,1,1,2,no_holiday,1994,movie,0,...,0,0,0,0,0,0,0,0,0,3.5
2,1,307,3,1,1,2,no_holiday,1993,movie,0,...,0,0,0,0,0,0,0,0,0,5.0
3,1,665,3,1,1,3,no_holiday,1995,movie,0,...,0,0,0,0,0,0,0,1,0,5.0
4,1,899,3,1,1,2,no_holiday,1952,movie,0,...,0,0,1,0,0,0,0,0,0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24983464,162541,50872,2,1,1,4,no_holiday,2007,movie,0,...,0,0,0,0,0,0,0,0,0,4.5
24983465,162541,55768,2,1,1,4,no_holiday,2007,movie,0,...,0,0,0,0,0,0,0,0,0,2.5
24983466,162541,56176,2,1,1,4,no_holiday,2007,movie,0,...,0,0,0,0,0,0,0,0,0,2.0
24983467,162541,58559,2,1,1,4,no_holiday,2008,movie,0,...,0,0,0,0,0,0,0,0,0,4.0


In [6]:
# Label encode categorical columns
categorical_columns = ['holiday', 'titleType', 'directors', 'actor']
label_encoders = {}

for column in categorical_columns:
    if column in added_imdb_context.columns.to_list():
        print(f"Column {column} is in context_data")
        label_encoder = LabelEncoder()
        added_imdb_context[column] = label_encoder.fit_transform(added_imdb_context[column])
        label_encoders[column] = label_encoder

added_imdb_context

Column holiday is in context_data
Column titleType is in context_data
Column directors is in context_data
Column actor is in context_data


,userId,movieId,day,isWeekday,season,partOfDay,holiday,movieYear,titleType,isAdult,...,genreNews,genreReality-tv,genreRomance,genreSci-fi,genreShort,genreSport,genreThriller,genreWar,genreWestern,rating
0,1,296,3,1,1,3,2,1994,0,0,...,0,0,0,0,0,0,1,0,0,5.0
1,1,306,3,1,1,2,2,1994,0,0,...,0,0,0,0,0,0,0,0,0,3.5
2,1,307,3,1,1,2,2,1993,0,0,...,0,0,0,0,0,0,0,0,0,5.0
3,1,665,3,1,1,3,2,1995,0,0,...,0,0,0,0,0,0,0,1,0,5.0
4,1,899,3,1,1,2,2,1952,0,0,...,0,0,1,0,0,0,0,0,0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24983464,162541,50872,2,1,1,4,2,2007,0,0,...,0,0,0,0,0,0,0,0,0,4.5
24983465,162541,55768,2,1,1,4,2,2007,0,0,...,0,0,0,0,0,0,0,0,0,2.5
24983466,162541,56176,2,1,1,4,2,2007,0,0,...,0,0,0,0,0,0,0,0,0,2.0
24983467,162541,58559,2,1,1,4,2,2008,0,0,...,0,0,0,0,0,0,0,0,0,4.0


In [7]:
train, temp = train_test_split(added_imdb_context, test_size=0.2, random_state=42)
del added_imdb_context
val, test = train_test_split(temp, test_size=0.5, random_state=42)
del temp

----

In [ ]:
train_target_ratings = train['rating'].values
val_target_ratings = val['rating'].values
test_target_ratings = test['rating'].values

train_data = train.drop(['rating'], axis=1)
val_data = val.drop(['rating'], axis=1)
test_data = test.drop(['rating'], axis=1)

In [ ]:
# # Standardize columns

scaler = MaxAbsScaler()
# target_scaler = MaxAbsScaler()

train_data = scaler.fit_transform(train_data)
val_data = scaler.fit_transform(val_data)
test_data = scaler.fit_transform(test_data)

# train_target_ratings = target_scaler.fit_transform(train_target_ratings.reshape(-1, 1))
# val_target_ratings = target_scaler.fit_transform(val_target_ratings.reshape(-1, 1))
# test_target_ratings = target_scaler.fit_transform(test_target_ratings.reshape(-1, 1))

In [ ]:
# Save the data to disk

training_data_path = config['training_data']
data_group = 'data_25m/'
folder = 'added_imdb_context/'

dump(scaler, file_depth + training_data_path + f"{data_group}{folder}25m_added_imdb_context_scaler.pkl")
# dump(target_scaler, file_depth + training_data_path + f"{data_group}{folder}25m_added_imdb_context_target_scaler.pkl")

for column in categorical_columns:
    try:
        if label_encoders[column] != {}:
            dump(label_encoders[column], file_depth + training_data_path + f"{data_group}{folder}{column}_label_encoder.pkl")
    except KeyError:
        pass
    

np.save(file_depth + training_data_path + f"{data_group}{folder}train_data.npy", train_data)
np.save(file_depth + training_data_path + f"{data_group}{folder}val_data.npy", val_data)
np.save(file_depth + training_data_path + f"{data_group}{folder}test_data.npy", test_data)
np.save(file_depth + training_data_path + f"{data_group}{folder}train_target_ratings.npy", train_target_ratings)
np.save(file_depth + training_data_path + f"{data_group}{folder}val_target_ratings.npy", val_target_ratings)
np.save(file_depth + training_data_path + f"{data_group}{folder}test_target_ratings.npy", test_target_ratings)